In [1]:
import os
from datasets import load_dataset

from ragas import EvaluationDataset, evaluate
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, SemanticSimilarity
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import HuggingfaceEmbeddings
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from dotenv import load_dotenv

load_dotenv()


evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini", temperature=0))
evaluator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-large"))


g:\anaconda\anaconda3\envs\chatbot-agent\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

dataset = load_dataset(
    "json",
    data_files="../data/eval/test_dataset_new.json",
    field='data',
    split='train'
)

eval_dataset = EvaluationDataset.from_hf_dataset(dataset)



In [3]:
eval_dataset[0]

SingleTurnSample(user_input='Đối tượng được dự thi tiếng Anh nội bộ', retrieved_contexts=['0\n\n      BỘ GIÁO DỤC VÀ ĐÀO TẠO                       CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM\n    ĐẠI HỌC BÁCH KHOA HÀ NỘI                                Độc lập – Tự do – Hạnh phúc\n           Số:        /QĐ-ĐHBK                                  Hà Nội, ngày        tháng 4 năm 2023\n                                             QUYẾT ĐỊNH\n                    Về việc Ban hành Quy định tổ chức thi tiếng Anh nội bộ\n                                    của Đại học Bách khoa Hà Nội', 'Điều 1. Phạm vi điều chỉnh và đối tượng áp dụng\n      Văn bản này quy định về công tác tổ chức thi tiếng Anh nội bộ Đại học Bách khoa\nHà Nội.\n      Đối tượng dự thi: Sinh viên Đại học Bách khoa Hà Nội.\n      Điều 2. Hội đồng thi và các tiểu ban\n      1. Hội đồng thi\n      a) Giám đốc ra quyết định thành lập Hội đồng thi (HĐT) để điều hành các công\nviệc liên quan đến công tác tổ chức các kỳ thi tiếng Anh nội bộ theo

In [4]:
import nest_asyncio
nest_asyncio.apply()

metrics = [
    # LLMContextRecall(llm=evaluator_llm), 
    FactualCorrectness(llm=evaluator_llm), 
    Faithfulness(llm=evaluator_llm),
    SemanticSimilarity(embeddings=evaluator_embeddings)
]

results = evaluate(dataset=eval_dataset, metrics=metrics)

Evaluating: 100%|██████████| 378/378 [03:55<00:00,  1.60it/s]


In [5]:
df = results.to_pandas()

In [6]:
df.head()

,user_input,retrieved_contexts,response,reference,factual_correctness,faithfulness,semantic_similarity
0,Đối tượng được dự thi tiếng Anh nội bộ,[0\n\n BỘ GIÁO DỤC VÀ ĐÀO TẠO ...,"Theo quy định tại Điều 1 của Văn bản này, đối ...",Đối tượng được dự thi tiếng Anh nội bộ là toàn...,0.67,1.000000,0.849053
1,Hội đồng thi tiếng Anh nội bộ gồm những ai,[0\n\n BỘ GIÁO DỤC VÀ ĐÀO TẠO ...,Hội đồng thi tiếng Anh nội bộ (HĐT) bao gồm Ch...,Hội đồng thi gồm:\r\n- Giám đốc ra quyết định ...,0.86,0.800000,0.832095
2,Các tiểu ban tổ chức thi tiếng Anh nội bộ,[Điều 1. Phạm vi điều chỉnh và đối tượng áp dụ...,"Theo quy định tại Điều 2 của văn bản này, các ...",Chủ tịch HĐT ra quyết định thành lập các tiểu ...,0.20,1.000000,0.749939
3,Nhiệm vụ của các tiểu ban trong tổ chức tiếng ...,[Điều 1. Phạm vi điều chỉnh và đối tượng áp dụ...,Các nhiệm vụ của các tiểu ban trong tổ chức ti...,- Tiểu ban Đề thi có nhiệm vụ tư vấn và tổ chứ...,0.76,0.882353,0.844743
4,Vai trò trong công tác tổ chức thi của các bên,[Điều 1. Phạm vi điều chỉnh và đối tượng áp dụ...,"Theo quy định tại Điều 2, các bên có vai trò s...",a) Tiểu ban Coi thi – Tổ chức thi phối hợp với...,0.32,1.000000,0.733865


In [7]:

results.upload()

Evaluation results uploaded! View at https://app.ragas.io/dashboard/alignment/evaluation/c234ce1e-aa6b-418e-bd38-6482d09895f7


'https://app.ragas.io/dashboard/alignment/evaluation/c234ce1e-aa6b-418e-bd38-6482d09895f7'